In [104]:
%pylab inline
from snakebite.client import Client
import pyaml as yaml
import sys
import pythonrun
reload(pythonrun)
import os
import seaborn as sns
from sklearn import metrics
import scala_python_endive_wrapper
reload(scala_python_endive_wrapper)
from scala_python_endive_wrapper import *
import pandas as pd
import os
from sklearn.cluster import KMeans

Populating the interactive namespace from numpy and matplotlib


In [82]:
hdfsclient = Client("amp-spark-master.amp", 8020, use_trash=False)
notebook_stdout = sys.stdout
notebook_stderr = sys.stderr
terminal_stdout = open('/dev/stdout', 'w')
terminal_stderr = open('/dev/stderr', 'w')
logpath = "/tmp/log"

In [61]:
executor_mem = "150g"
cores_per_executor = 32
num_executors = 14
EGR1_PATH = '/user/vaishaal/endive-data/EGR1'

In [106]:
# this is where the magic happens
results = [] 
num_filters = 256
gamma = 0.1
res = run_kitchensink_featurize_pipeline(EGR1_PATH,
                           "/tmp/filters.csv", 
                           logpath, 
                           seed=0,
                           filter_gen_gen=make_gaussian_filter_gen,
                           cores_per_executor=cores_per_executor,
                           num_filters=num_filters,
                           sample=0.001,
                           alphabet_size=1,
                           kmer_size=64,
                           gamma=gamma,
                           executor_mem=executor_mem,
                           num_partitions=cores_per_executor*num_executors,
                           featuresOutput="/user/vaishaal/featurizedWindows/sequence-dnase-naive/tiny/{0}-filters/{1}-gamma/replicate-0".format(num_filters, gamma),
                           num_executors=num_executors)
results.append(res)

ALPHABET SIZE  1
(1024, 64)
{'kmerLength': 64, 'reference': '/home/eecs/akmorrow/ADAM/endive/workfiles/hg19.2bit', 'filtersPath': '/tmp/filters.csv', 'numPartitions': 448, 'dnaseBams': '/data/anv/DREAMDATA/dnase_bams/merged_coverage/', 'aggregatedSequenceOutput': '/user/vaishaal/endive-data/EGR1', 'featuresOutput': '/user/vaishaal/featurizedWindows/sequence-dnase-naive/tiny/1024-filters/0.1-gamma/replicate-0', 'seed': 0, 'dnaseNarrow': '/data/anv/DREAMDATA/DNASE/peaks/relaxed/', 'approxDim': 1024, 'featurizeSample': 0.001, 'alphabetSize': 1, 'readFiltersFromDisk': True}


In [62]:
chromosomes = map(lambda x: "chr{0}".format(x+1), range(23))
# CHANGE ME BASED ON ENUM + TF
egr1_cell_types = string_to_enum_celltypes(['GM12878', 'H1hESC', 'HCT116', 'MCF7'])

In [ ]:
results_df = cross_validate("/user/vaishaal/featurizedWindows/sequence-dnase-naive/tiny/{0}-filters/{1}-gamma/replicate-0".format(num_filters, gamma), 
                            hdfsclient, 
                            chromosomes, 
                            egr1_cell_types,
                            numHoldOutChr=1, 
                            numHoldOutCell=1,
                            cores_per_executor = 32,
                            num_executors = 14,
                            executor_mem="150g",
                            num_folds=1, 
                            negativeSamplingFreqs=[0.001, 0.0001],
                            regs=[1e-2])
    

FOLD 0
HOLDING OUT CHROMOSOMES ['chr12']
HOLDING OUT CELL TYPES [3]
RUNING SOLVER WITH REG=0.01


In [68]:
old = results_df

In [84]:
results_df

,negativeSamplingFreq,reg,test_celltypes,test_chromosomes,train_auPRC,train_auROC,train_recall_at_05_fdr,train_recall_at_10_fdr,train_recall_at_25_fdr,train_recall_at_50_fdr,val_auPRC,val_auROC,val_recall_at_05_fdr,val_recall_at_10_fdr,val_recall_at_25_fdr,val_recall_at_50_fdr
0,0.0010,0.01,[3],[chr12],0.954726,0.969473,0.736193,0.856400,0.982679,0.998876,0.035594,0.903343,0,0,0,0
1,0.0001,0.01,[3],[chr12],0.993585,0.994914,0.998978,0.999171,0.999695,1.000000,0.017933,0.884693,0,0,0,0
